# A/B Testing Simulation

In this notebook, a user has a hidden preference within a single query. We use this to explore A/B testing. 

Of course, this problem needs to be multiplied over millions of queries!

1. The last judgments from chapter 11
2. Fully train a model w/ two feature sets (turn ch 11 auto LTR notebook into function) 
3. Simulate user interaction w/ rankings

In [1]:
import numpy as np
import pandas as pd
import random; random.seed(0)
import glob

import requests
import sys
sys.path.append('..')
from aips import *
from ltr.client.solr_client import SolrClient

client = SolrClient(solr_base=SOLR_URL)

In [2]:
def all_sessions():
    sessions = pd.concat([pd.read_csv(f, compression='gzip')
                          for f in glob.glob('retrotech/sessions/*_sessions.gz')])
    return sessions.rename(columns={'clicked_doc_id': 'doc_id'})
    
sessions = all_sessions()
sessions

,sess_id,query,rank,doc_id,clicked
0,50002,blue ray,0.0,600603141003,True
1,50002,blue ray,1.0,827396513927,False
2,50002,blue ray,2.0,24543672067,False
3,50002,blue ray,3.0,719192580374,False
4,50002,blue ray,4.0,885170033412,True
...,...,...,...,...,...
74995,5001,transformers dark of the moon,10.0,47875841369,False
74996,5001,transformers dark of the moon,11.0,97363560449,False
74997,5001,transformers dark of the moon,12.0,93624956037,False
74998,5001,transformers dark of the moon,13.0,97363532149,False


In [3]:
sessions['query'].unique()

array(['blue ray', 'bluray', 'dryer', 'headphones', 'ipad', 'iphone',
       'kindle', 'lcd tv', 'macbook', 'nook', 'star trek', 'star wars',
       'transformers dark of the moon'], dtype=object)

In [4]:
new_sessions = sessions[sessions['query'] == 'macbook'].copy() 

In [5]:
random.seed(0)

# Make two queries identical, except for the query
# TODO? Randomly flip some of the clicked bools, but this might make it non deterministic
def copy_query_sessions(sessions, src_query, dest_query):
    new_sessions = sessions[sessions['query'] == src_query].copy()  
    new_sessions['draw'] = np.random.rand(len(new_sessions), 1)
    # unclick some in the new query for a bit of noise
    new_sessions[new_sessions['clicked'] & (new_sessions['draw'] < 0.04)]['clicked'] = False
    new_sessions['query'] = dest_query
    return pd.concat([sessions, new_sessions.drop('draw', axis=1)])

sessions = copy_query_sessions(sessions, 'transformers dark of the moon', 'transformers dark of moon')
sessions = copy_query_sessions(sessions, 'transformers dark of the moon', 'dark of moon')
sessions = copy_query_sessions(sessions, 'transformers dark of the moon', 'dark of the moon')
sessions = copy_query_sessions(sessions, 'headphones', 'head phones')
sessions = copy_query_sessions(sessions, 'lcd tv', 'lcd television')
sessions = copy_query_sessions(sessions, 'lcd tv', 'television, lcd')
sessions = copy_query_sessions(sessions, 'macbook', 'apple laptop')
sessions = copy_query_sessions(sessions, 'iphone', 'apple iphone')
sessions = copy_query_sessions(sessions, 'kindle', 'amazon kindle')
sessions = copy_query_sessions(sessions, 'kindle', 'amazon ereader')
sessions = copy_query_sessions(sessions, 'blue ray', 'blueray')

sessions

/tmp/ipykernel_29/4270496349.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_29/4270496349.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_29/4270496349.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_29/4270496

,sess_id,query,rank,doc_id,clicked
0,50002,blue ray,0.0,600603141003,True
1,50002,blue ray,1.0,827396513927,False
2,50002,blue ray,2.0,24543672067,False
3,50002,blue ray,3.0,719192580374,False
4,50002,blue ray,4.0,885170033412,True
...,...,...,...,...,...
149995,55001,blueray,25.0,22265004517,False
149996,55001,blueray,26.0,885170038875,False
149997,55001,blueray,27.0,786936817232,False
149998,55001,blueray,28.0,600603132872,False


In [6]:
sessions['query'].unique()

array(['blue ray', 'bluray', 'dryer', 'headphones', 'ipad', 'iphone',
       'kindle', 'lcd tv', 'macbook', 'nook', 'star trek', 'star wars',
       'transformers dark of the moon', 'transformers dark of moon',
       'dark of moon', 'dark of the moon', 'head phones',
       'lcd television', 'television, lcd', 'apple laptop',
       'apple iphone', 'amazon kindle', 'amazon ereader', 'blueray'],
      dtype=object)

## Inject bias for transformers / transformers dvd

This simulates biased sessions in the data, as if the user never actually sees (and hence never clicks) their actual desired item. If the users desired results are shown, those results get a higher probability of click. Otherwise there is a lower probability of clicks.

In [7]:
next_sess_id = sessions['sess_id'].max()

# For some reason, the sessions only capture examines on the 'dubbed' transformers movies
# ie the Japanese shows brought to an English-speaking market. But we'll see this is not what the 
# user wants (ie presentation bias). These are 'meh' mildly interesting. There are also many many
# completely irrelevant movies.

# What the user wants, but never visible! Never gets clicked!
# These are the widescreen transformers dvds of the hollywood movies
desired_movies = ["97360724240", "97360722345", "97368920347"] 

# Bunch of random merchandise
irrelevant_transformers_products = ["708056579739", "93624995012", "47875819733", "47875839090", "708056579746",
                                     "47875332911", "47875842328", "879862003524", "879862003517", "93624974918",
                                     ] 

# Other transformer movies
meh_transformers_movies = ["97363455349", "97361312743", "97361372389", "97361312804", "97363532149", "97363560449"]

displayed_transformer_products = meh_transformers_movies + irrelevant_transformers_products

new_sessions = []
for i in range(0,5000):
    random.shuffle(displayed_transformer_products)

    # shuffle each session
    for rank, upc in enumerate(displayed_transformer_products):
        clicked = False
        draw = random.random()

        if upc in meh_transformers_movies:
            if draw < 0.13:
                clicked = True
        elif upc in irrelevant_transformers_products:
            if draw < 0.005:
                clicked = True
        elif upc in desired_transformers_movies:
            if draw < 0.65:
                clicked = True

        new_sessions.append({'sess_id': next_sess_id + i, 
                             'query': 'transformers dvd', 
                             'rank': rank,
                             'clicked': clicked,
                             'doc_id': upc
                             })


sessions = sessions.append(new_sessions)

/tmp/ipykernel_29/2230816581.py:49: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



## Chapter 11 In One Function (omitted) 

Wrapping up Chapter 11 in a single function `sessions_to_sdbn`

In [8]:
def sessions_to_sdbn(sessions, prior_weight=10, prior_grade=0.2) -> pd.DataFrame:
    """ Compute SDBN of the provided query as a dataframe.
        Where we left off at end of 'overcoming confidence bias' 
        """
    all_sdbn = pd.DataFrame()
    for query in sessions['query'].unique():
        sdbn_sessions = sessions[sessions['query'] == query].copy().set_index('sess_id')

        last_click_per_session = sdbn_sessions.groupby(['clicked', 'sess_id'])['rank'].max()[True]

        sdbn_sessions['last_click_rank'] = last_click_per_session
        sdbn_sessions['examined'] = sdbn_sessions['rank'] <= sdbn_sessions['last_click_rank']

        sdbn = sdbn_sessions[sdbn_sessions['examined']].groupby('doc_id')[['clicked', 'examined']].sum()
        sdbn['grade'] = sdbn['clicked'] / sdbn['examined']
        sdbn['query'] = query

        sdbn = sdbn.sort_values('grade', ascending=False)

        sdbn['prior_a'] = prior_grade*prior_weight
        sdbn['prior_b'] = (1-prior_grade)*prior_weight

        sdbn['posterior_a'] = sdbn['prior_a'] +  sdbn['clicked']
        sdbn['posterior_b'] = sdbn['prior_b'] + (sdbn['examined'] - sdbn['clicked'])

        sdbn['beta_grade'] = sdbn['posterior_a'] / (sdbn['posterior_a'] + sdbn['posterior_b'])

        sdbn.sort_values('beta_grade', ascending=False)
        all_sdbn = all_sdbn.append(sdbn)
    return all_sdbn[['query', 'clicked', 'examined', 'grade', 'beta_grade']].reset_index().set_index(['query', 'doc_id'])

queries = ['dryer', 'bluray', 'blue ray', 'headphones', 'ipad', 'iphone',
           'kindle', 'lcd tv', 'macbook', 'nook', 'star trek', 'star wars',
           'transformers dark of the moon']



## Listing 12.1 Use Convert Raw Sessions to SDBN

In this listing we user our "chapter 11 in one function" `sessions_to_sdbn` to rebuild training data.

In [9]:
sdbn = sessions_to_sdbn(sessions,
                        prior_weight=10,
                        prior_grade=0.2)
sdbn

/tmp/ipykernel_29/3617411679.py:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_29/3617411679.py:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_29/3617411679.py:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_29/3617411679.py:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_29/3617411679.py:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_29/3617411679.py:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

clicked  examined     grade  beta_grade
query            doc_id                                               
blue ray         27242815414        42        42  1.000000    0.846154
                 600603132872       46        88  0.522727    0.489796
                 827396513927     1304      3381  0.385685    0.385137
                 600603141003      978      2620  0.373282    0.372624
                 885170033412      568      2184  0.260073    0.259799
...                                ...       ...       ...         ...
transformers dvd 47875819733        24      1679  0.014294    0.015394
                 708056579739       23      1659  0.013864    0.014979
                 879862003524       23      1685  0.013650    0.014749
                 93624974918        19      1653  0.011494    0.012628
                 47875839090        16      1669  0.009587    0.010721

[649 rows x 4 columns]

## Chapter 10 Functions (omitted from book)

All of the following are support functions for the chapter:

1. Convert the sdbn dataframe into individual `Judgment` objects needed for training the model from chapter 10
2. Pairwise transformation of the data
3. Normalization of the data
4. Training the model
5. Uploading the model to Solr

All of these steps are covered in Chapter 10.

In [10]:
import requests
import numpy as np
from ltr.judgments import judgments_from_file, judgments_to_nparray
from sklearn import svm
import json
import math
from itertools import groupby
from ltr.log import FeatureLogger
from ltr.judgments import judgments_open
from itertools import groupby
from ltr import download
from ltr.judgments import judgments_writer

from ltr.judgments import Judgment

def sdbn_to_judgments(sdbn):
    """Turn pandas dataframe into ltr judgments objects."""
    judgments = []
    queries = {}
    next_qid = 0
    for row_dict in sdbn.reset_index().to_dict(orient="records"):
        # Round grade to 10ths, Map 0.3 -> 3, etc
        grade = round(row_dict['beta_grade'], 1) * 10
        qid = -1
        if row_dict['query'] in queries:
            qid = queries[row_dict['query']]
        else:
            queries[row_dict['query']] = next_qid
            qid = next_qid
            next_qid += 1
        assert qid != -1
        
        judgments.append(Judgment(doc_id=row_dict['doc_id'],
                                  keywords=row_dict['query'],
                                  qid=qid,
                                  grade=int(grade))
                        )
    return judgments


sdbn_to_judgments(sdbn)


def write_judgments(judgments, dest='retrotech_judgments.txt'):
    with judgments_writer(open(dest, 'wt')) as writer:
        for judgment in judgments:
            writer.write(judgment)
            
write_judgments(sdbn_to_judgments(sdbn))
!cat retrotech_judgments.txt


def normalize_features(logged_judgments):
    all_features = []
    means = [0] * len(logged_judgments[0].features)
    for judgment in logged_judgments:
        for idx, f in enumerate(judgment.features):
            means[idx] += f
        all_features.append(judgment.features)
    
    for i in range(len(means)):
        means[i] /= len(logged_judgments)
      
    std_devs = [0.0] * len(logged_judgments[0].features)
    for judgment in logged_judgments:
        for idx, f in enumerate(judgment.features):
            std_devs[idx] += (f - means[idx])**2
            
    for i in range(len(std_devs)):
        std_devs[i] /= len(logged_judgments)
        std_devs[i] = math.sqrt(std_devs[i])
        
    # Normalize!
    normed_judgments = []
    for judgment in logged_judgments:
        normed_features = [0.0] * len(judgment.features)
        for idx, f in enumerate(judgment.features):
            normed = 0.0
            if std_devs[idx] > 0: 
                normed = (f - means[idx]) / std_devs[idx]
            normed_features[idx] = normed
        normed_judgment=Judgment(qid=judgment.qid,
                                 keywords=judgment.keywords,
                                 doc_id=judgment.doc_id,
                                 grade=judgment.grade,
                                 features=normed_features)
        normed_judgment.old_features=judgment.features
        normed_judgments.append(normed_judgment)

    return means, std_devs, normed_judgments


def pairwise_transform(normed_judgments, weigh_difference = True):
        
    predictor_deltas = []
    feature_deltas = []
    
    # For each query's judgments
    for qid, query_judgments in groupby(normed_judgments, key=lambda j: j.qid):

        # Annoying issue consuming python iterators, we ensure we have two
        # full copies of each query's judgments
        query_judgments_copy_1 = list(query_judgments) 
        query_judgments_copy_2 = list(query_judgments_copy_1)

        # Examine every judgment combo for this query, 
        # if they're different, store the pairwise difference:
        # +1 if judgment1 more relevant
        # -1 if judgment2 more relevant
        for judgment1 in query_judgments_copy_1:
            for judgment2 in query_judgments_copy_2:
                
                j1_features=np.array(judgment1.features)
                j2_features=np.array(judgment2.features)
                
                if judgment1.grade > judgment2.grade:
                    diff = judgment1.grade - judgment2.grade if weigh_difference else 1.0
                    predictor_deltas.append(+1)
                    feature_deltas.append(diff * (j1_features-j2_features))
                elif judgment1.grade < judgment2.grade:
                    diff = judgment2.grade - judgment1.grade if weigh_difference else 1.0
                    predictor_deltas.append(-1)
                    feature_deltas.append(diff * (j1_features-j2_features))

    # For training purposes, we return these as numpy arrays
    return np.array(feature_deltas), np.array(predictor_deltas)

def upload_model(model, model_name, means, std_devs, feature_set):

    linear_model = {
      "store": "test",
      "class": "org.apache.solr.ltr.model.LinearModel",
      "name": model_name,
      "features": [
      ],
      "params": {
          "weights": {
          }
      }
    }

    ftr_model = {}
    ftr_names = [ftr['name'] for ftr in feature_set]
    for idx, ftr_name in enumerate(ftr_names):
        config = {
            "name": ftr_name,
            "norm": {
                "class": "org.apache.solr.ltr.norm.StandardNormalizer",
                "params": {
                    "avg": str(means[idx]),
                    "std": str(std_devs[idx])
                }
            }
        }
        linear_model['features'].append(config)
        linear_model['params']['weights'][ftr_name] =  model.coef_[0][idx] 

    print(f"PUT {SOLR_URL}/solr/products/schema/model-store")
    print(json.dumps(linear_model, indent=2))

    # Delete old model
    resp = requests.delete(f"{SOLR_URL}/products/schema/model-store/test_model")


    # Upload the model
    resp = requests.put(f"{SOLR_URL}/products/schema/model-store", json=linear_model)
    resp.text
    
    
## TODO - can't easily to test/train split on these few queries
##   make more queries?

def ranksvm_ltr(sdbn, model_name, feature_set):
    """Train a RankSVM model via Solr, store in Solr."""
    judgments = sdbn_to_judgments(sdbn)
    judgments_path = 'retrotech_judgments.txt'
    write_judgments(judgments, judgments_path)
    
    # For more on this code, review Chapter 10
    requests.delete(f"{SOLR_URL}/products/schema/feature-store/test")
    
    resp = requests.put(f"{SOLR_URL}/products/schema/feature-store",
                    json=feature_set)

    ftr_logger=FeatureLogger(client, index='products', feature_set='test', id_field='upc')

    with judgments_open(judgments_path) as judgment_list:
        for qid, query_judgments in groupby(judgments, key=lambda j: j.qid):
            ftr_logger.log_for_qid(judgments=query_judgments, 
                                   qid=qid,
                                   keywords=judgment_list.keywords(qid))

    logged_judgments = ftr_logger.logged
    means, std_devs, normed_judgments = normalize_features(logged_judgments)
    feature_deltas, predictor_deltas = pairwise_transform(normed_judgments)

    model = svm.LinearSVC(max_iter=10000, verbose=1)
    model.fit(feature_deltas, predictor_deltas)  
    upload_model(model, model_name, means, std_devs, feature_set)


# qid:0: blue ray*1
# qid:1: bluray*1
# qid:2: dryer*1
# qid:3: headphones*1
# qid:4: ipad*1
# qid:5: iphone*1
# qid:6: kindle*1
# qid:7: lcd tv*1
# qid:8: macbook*1
# qid:9: nook*1
# qid:10: star trek*1
# qid:11: star wars*1
# qid:12: transformers dark of the moon*1
# qid:13: transformers dark of moon*1
# qid:14: dark of moon*1
# qid:15: dark of the moon*1
# qid:16: head phones*1
# qid:17: lcd television*1
# qid:18: television, lcd*1
# qid:19: apple laptop*1
# qid:20: apple iphone*1
# qid:21: amazon kindle*1
# qid:22: amazon ereader*1
# qid:23: blueray*1
# qid:24: transformers dvd*1

8	qid:0	 # 27242815414	blue ray
5	qid:0	 # 600603132872	blue ray
4	qid:0	 # 827396513927	blue ray
4	qid:0	 # 600603141003	blue ray
3	qid:0	 # 885170033412	blue ray
3	qid:0	 # 883929140855	blue ray
2	qid:0	 # 24543672067	blue ray
2	qid:0	 # 813774010904	blue ray
2	qid:0	 # 36725617605	blue ray
2	qid:0	 # 786936817232	blue ray
2	qid:0	 # 36725608443	blue ray
2	qid:0	 # 719192580374	blue ray
2	qid:0	 # 25192

## Also Chapter 10 - Perform a test / train split on the SDBN data

This function is broken out from the model training. It lets us train a model on one set of data (reusing the chapter 10 training code), reserving test queries for evaluation.

In [11]:
from math import floor

def test_train_split(sdbn, train):
    """Split queries in sdbn into train / test split with `train` proportion going to training set."""
    queries = sdbn.index.get_level_values('query').unique().copy().tolist()
    random.shuffle(queries)
    num_queries = len(queries)
    split_point = floor(num_queries * train)
    
    train_queries = queries[:split_point]
    test_queries = queries[split_point:]
    return sdbn.loc[train_queries, :], sdbn.loc[test_queries]


## Chapter 10 - Search Code

Also from Chapter 10, a simple function to search using the LTR model and return a list of search results.

In [12]:
def search(query, model_name, at=10, log=False):
    """ Search using test_model LTR model (see rq to and qf params below). """
    fuzzy_kws = "~" + ' ~'.join(query.split())
    squeezed_kws = "".join(query.split())
    
    rq = \
        "{!ltr reRankDocs=60000 reRankWeight=10.0 model=" + model_name \
        + " efi.fuzzy_keywords=\"" + fuzzy_kws + "\" " \
        + "efi.squeezed_keywords=\"" + squeezed_kws +"\" " \
        + "efi.keywords=\"" + query + "\"}"

    request = {
            "fields": ["upc", "name", "manufacturer", "score"],
            "limit": at,
            "params": {
              "rq": rq,
              "qf": "name name_ngram upc manufacturer shortDescription longDescription",
              "defType": "edismax",
              "q": query
            }
        }
    
    if log:
        print(request)

    resp = requests.post(f"{SOLR_URL}/products/select", 
                                   json=request).json()
    
    if log:
        print(resp)
        
    search_results = resp['response']['docs']

    for rank, result in enumerate(search_results):
        result['rank'] = rank
        
    return search_results

def search_and_grade(query, model_name, sdbn, desired=[]):
    results = search(query, model_name, at=10)
    results = pd.DataFrame(results)
    results['desired'] = False
    for upc in desired:
        results.loc[results['upc'] == upc, 'desired'] = True
        
    sdbn_query = sdbn.loc[query].copy().reset_index()
    return results.merge(sdbn_query, left_on='upc', right_on='doc_id', how='left')

## Evaluate the model on the test set

This function computes the model's performance on a set of test queries.

In [13]:
def eval_model(test, model_name, sdbn, at=10):
    queries = test.index.get_level_values('query').unique()
    collection = "products"
    
    query_results = {}
    
    for query in queries:
        search_results = search(query, model_name, at=at)

        results = pd.DataFrame(search_results).reset_index()
        judgments = sdbn.loc[query, :].copy().reset_index()
        judgments['doc_id'] = judgments['doc_id'].astype(str)
        if len(results) == 0:
            print(f"No Results for {query}")
            query_results[query] = 0
        else:
            graded_results = results.merge(judgments, left_on='upc', right_on='doc_id', how='left')
            print(graded_results)
            graded_results[['clicked', 'examined', 'grade', 'beta_grade']] = graded_results[['clicked', 'examined', 'grade', 'beta_grade']].fillna(0)
            grade_results = graded_results.drop('doc_id', axis=1)

            query_results[query] = (graded_results['beta_grade'].sum() / at)
    return query_results

## Listing 12.2 - model training

We wrap all the important decisions from chapter 10 in a few lines 

In [16]:
random.seed(1234)

feature_set = [
    {
      "name" : "long_description_bm25",
      "store": "test",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "longDescription:(${keywords})"
      }
    },
    {
      "name" : "short_description_constant",
      "store": "test",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "shortDescription:(${keywords})^=1"
      }
    }
]

train, test = test_train_split(sdbn, train=0.8)
ranksvm_ltr(train, model_name='test1', feature_set=feature_set)
eval_model(test, model_name='test1', sdbn=sdbn)

{!terms f=upc}36725236271,883393003458,36725234789,882777064009,22265004289,27242817197,729507810218,885170042704,812491010310,812491010334,827912072969,74000373105,884483335329,827912068467,827912068474,97278016000,696211503197,605342041546,882777064207,885170042667,13803112610,729507813059,22265004258,719192579996,600603139758,36725235564,22265004302,723755834491
Searching products [Status: 200]
Missing doc 600603139758
{!terms f=upc}885909457588,885909472376,821793013776,635753493559,722868830062,885909471812,92636260712,886111271283,885909457632,885909393404,885909457601,716829772249,885909457595,600603132827,27242798236,884962753071,886111287055,635753490879,843404073153,610839379408
Searching products [Status: 200]
Missing doc 600603132827
{!terms f=upc}27242815414,600603132872,827396513927,600603141003,885170033412,883929140855,24543672067,813774010904,36725617605,786936817232,36725608443,719192580374,25192073007,75993997675,36725608894,786936817218,711719983156,22265052211,8839

{'dryer': 0.0,
 'blue ray': 0.0,
 'headphones': 0.0,
 'dark of moon': 0.0,
 'transformers dvd': 0.003258006235976338}

In [17]:
# # What the user wants, but never visible! Never gets clicked!
# These are the widescreen transformers dvds of the hollywood movies
desired_movies = ["97360724240", "97360722345", "97368920347"] 
result = search_and_grade('transformers dvd', 'test1', sdbn, desired_movies)
upcs1 = result['upc']
result

,upc,name,manufacturer,score,rank,desired,doc_id,clicked,examined,grade,beta_grade
0,708056579746,Nintendo - Transformers 3 Stylus 2-Pack,Nintendo,0.070327,0,False,708056579746,26.0,1664.0,0.015625,0.016726
1,47875332911,Transformers: Revenge of the Fallen - Windows,Activision,0.062276,1,False,47875332911,24.0,1630.0,0.014724,0.015854
2,34707056190,Memorex - 50-Pack 16x DVD+R Disc Spindle,Memorex,0.061168,2,False,NaN,NaN,NaN,NaN,NaN
3,23942950585,Verbatim - 25-Pack 16x DVD-R Disc Spindle,Verbatim,0.059489,3,False,NaN,NaN,NaN,NaN,NaN
4,659846419028,Digital Innovations - DvdDr Laser Lens Cleaner...,Digital Innovations,0.058364,4,False,NaN,NaN,NaN,NaN,NaN
5,659846419035,Digital Innovations - CleanDr. Laser Lens Clea...,Digital Innovations,0.058153,5,False,NaN,NaN,NaN,NaN,NaN
6,85854103756,Case Logic - 200-Disc Expandable DVD Album - B...,Case Logic,0.057207,6,False,NaN,NaN,NaN,NaN,NaN
7,716829999523,Coby - Portable DVD Player with Dual TFT-LCD S...,Coby,0.056768,7,False,NaN,NaN,NaN,NaN,NaN
8,600603124068,Init&#x2122; - 24-Disc CD/DVD Wallet - Red,Init&#x99;,0.056362,8,False,NaN,NaN,NaN,NaN,NaN
9,34707056398,Memorex - 50-Pack 16x DVD-R Disc Spindle,Memorex,0.056092,9,False,NaN,NaN,NaN,NaN,NaN


## Listing 12.3

Train a model that performs better offline called `test2`

In [18]:
random.seed(1234)


feature_set_better = [
    {
      "name" : "name_fuzzy",
      "store": "test",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : { 
        "q" : "name_ngram:(${keywords})"
      }
    },
    {
      "name" : "name_pf2",
      "store": "test",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : { 
        "q" : "{!edismax qf=name name pf2=name}(${keywords})"
      }
    },
    {
      "name" : "shortDescription_pf2",
      "store": "test",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : { 
        "q" : "{!edismax qf=shortDescription pf2=shortDescription}(${keywords})"
      }
    },
]

sdbn = sessions_to_sdbn(sessions) # chapter 11: generate training data

train, test = test_train_split(sdbn, train=0.8)
ranksvm_ltr(train, 'test2', feature_set_better) # chapter 10: train the model -> the 'LTR engine'
eval2 = eval_model(test, 'test2', sdbn)

eval2

/tmp/ipykernel_29/3617411679.py:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_29/3617411679.py:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_29/3617411679.py:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_29/3617411679.py:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_29/3617411679.py:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_29/3617411679.py:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

{!terms f=upc}36725236271,883393003458,36725234789,882777064009,22265004289,27242817197,729507810218,885170042704,812491010310,812491010334,827912072969,74000373105,884483335329,827912068467,827912068474,97278016000,696211503197,605342041546,882777064207,885170042667,13803112610,729507813059,22265004258,719192579996,600603139758,36725235564,22265004302,723755834491
Searching products [Status: 200]
Missing doc 600603139758
{!terms f=upc}885909457588,885909472376,821793013776,635753493559,722868830062,885909471812,92636260712,886111271283,885909457632,885909393404,885909457601,716829772249,885909457595,600603132827,27242798236,884962753071,886111287055,635753490879,843404073153,610839379408
Searching products [Status: 200]
Missing doc 600603132827
{!terms f=upc}27242815414,600603132872,827396513927,600603141003,885170033412,883929140855,24543672067,813774010904,36725617605,786936817232,36725608443,719192580374,25192073007,75993997675,36725608894,786936817218,711719983156,22265052211,8839

/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



   index           upc                                               name  \
0      0   13132318394                      Blue Valentine - Blu-ray Disc   
1      1  603497983438  Blue Man Group: How to Be a Megastar Live! - B...   
2      2   43396263635  Blue Streak - Widescreen Dubbed Subtitle AC3 -...   
3      3  741952696094  Rookie Blue: The Complete First Season (4 Disc...   
4      4   43396349827  Blue Streak & National Security (2 Disc) - 2 P...   
5      5   25192101021  Blue Crush 2 - Widescreen Dubbed Subtitle AC3 ...   
6      6   14381614954  Powder Blue - Widescreen Subtitle AC3 Dolby Dt...   
7      7  883929118670  Deep Blue Sea - Widescreen Subtitle AC3 Dolby ...   
8      8   95121534770                  Sportline - Pedometer - Gray/Blue   
9      9  883929137114                                       Blu-ray Disc   

  manufacturer     score  rank doc_id  clicked  examined  grade  beta_grade  
0           \N  0.052326     0    NaN      NaN       NaN    NaN         Na

{'dryer': 0.07068309073137659,
 'blue ray': 0.0,
 'headphones': 0.1285279187817259,
 'dark of moon': 0.41539851567397756,
 'transformers dvd': 0.10077083021678328}

In [19]:
# # What the user wants, but never visible! Never gets clicked!
# These are the widescreen transformers dvds of the hollywood movies
desired_movies = ['97360724240', '97363560449', '97363532149', '97360810042']
result = search_and_grade('transformers dvd', 'test2', sdbn, desired_movies)
upcs2 = result['upc']
result

,upc,name,manufacturer,score,rank,desired,doc_id,clicked,examined,grade,beta_grade
0,32429037763,Transformers - DVD,\N,0.223765,0,False,NaN,NaN,NaN,NaN,NaN
1,97368920347,The Transformers: The Movie - DVD,\N,0.050765,1,False,NaN,NaN,NaN,NaN,NaN
2,826663126044,Transformers Japanese Collection: Headmasters ...,\N,0.043840,2,False,NaN,NaN,NaN,NaN,NaN
3,826663114218,"Transformers: Season 2, Vol. 1 - DVD",\N,0.043216,3,False,NaN,NaN,NaN,NaN,NaN
4,97037110192,"Transformers: Serie Megatron, Vol. 1 - DVD",\N,0.041312,4,False,NaN,NaN,NaN,NaN,NaN
5,826663129892,Transformers Prime: Darkness Rising - Fullscre...,\N,0.038296,5,False,NaN,NaN,NaN,NaN,NaN
6,97363455349,Transformers - Widescreen Dubbed Subtitle AC3 ...,\N,0.038296,6,False,97363455349,664.0,1937.0,0.342798,0.342065
7,97361312743,Transformers - Widescreen Dubbed Subtitle AC3 ...,\N,0.038296,7,False,97361312743,657.0,1916.0,0.342902,0.342160
8,97361372389,Transformers - Widescreen Dubbed Subtitle AC3 ...,\N,0.038296,8,False,97361372389,622.0,1919.0,0.324127,0.323484
9,400173151118,Transformers Cybertron The Ultimate Collection...,\N,0.037574,9,False,NaN,NaN,NaN,NaN,NaN


In [20]:
def live_user_query(query, model_name,
                    desired, meh,
                    desired_prob=0.15, 
                    meh_prob=0.03, 
                    uninteresting_prob=0.01,
                    quit_per_rank_prob=0.2):
    """Live user for 'query' where purchase probability depends on if 
       products upc is in one of three sets.
       
       Users purchase a single product per session.    
       
       Users quit with `quit_per_rank_prod` after scanning each rank
       
       """   
    search_results = search(query, model_name, at=10)

    results = pd.DataFrame(search_results).reset_index()
    for doc in results.to_dict(orient="records"):
        clicked = False
        draw = random.random()
        
        upc = doc['upc']

        if upc in desired:
            if draw < desired_prob:
                return True
        elif upc in meh:
            if draw < meh_prob:
                return True
        else:
            if draw < uninteresting_prob:
                return True
            
        if random.random() < quit_per_rank_prob:
            return False
    
    return False


In [21]:
random.seed(1234)

wants_to_purchase = ['97360724240', '97363560449', '97363532149', '97360810042']
might_purchase = ['97361312743', '97363455349', '97361372389']

model1_purchases = 0
model2_purchases = 0

def a_or_b_model(query, a_model, b_model):
    """Randomly assign this user to a or b"""
    draw = random.random()
    
    user_made_purchase = False
    model_name = None
    if draw < 0.5:
        model_name=a_model
    else:
        model_name=b_model
        
    purchase_made = live_user_query(query=query, 
                                   model_name=model_name,
                                   desired=wants_to_purchase,
                                   meh=might_purchase)
    return (model_name, purchase_made)


NUM_USERS=1000
purchases = {'test1': 0, 'test2': 0}
for _ in range(0, NUM_USERS):
    
    model_name, purchase_made = a_or_b_model(query='transformers dvd', 
                                             a_model='test1',
                                             b_model='test2')
    if purchase_made:
        purchases[model_name]+= 1 
    
purchases

{'test1': 21, 'test2': 15}

In [22]:
model1_purchases, model2_purchases

(0, 0)

In [23]:
all_upcs = set(upcs1.tolist() + upcs2.tolist())
len(all_upcs)

20

In [24]:
upcs2.tolist()

['32429037763',
 '97368920347',
 '826663126044',
 '826663114218',
 '97037110192',
 '826663129892',
 '97363455349',
 '97361312743',
 '97361372389',
 '400173151118']

In [25]:
sessions['query'].unique()

array(['blue ray', 'bluray', 'dryer', 'headphones', 'ipad', 'iphone',
       'kindle', 'lcd tv', 'macbook', 'nook', 'star trek', 'star wars',
       'transformers dark of the moon', 'transformers dark of moon',
       'dark of moon', 'dark of the moon', 'head phones',
       'lcd television', 'television, lcd', 'apple laptop',
       'apple iphone', 'amazon kindle', 'amazon ereader', 'blueray',
       'transformers dvd'], dtype=object)

In [26]:
#1. Just use the queries we have to do a test/train split
#2. Simulate the user for A/B test

In [27]:
sessions[sessions['query'] == 'headphones']

,sess_id,query,rank,doc_id,clicked
0,30002,headphones,0.0,803238004525,True
1,30002,headphones,1.0,615104173552,False
2,30002,headphones,2.0,848447000135,False
3,30002,headphones,3.0,27242807785,False
4,30002,headphones,4.0,878615035287,False
...,...,...,...,...,...
149995,35001,headphones,25.0,27242798236,False
149996,35001,headphones,26.0,709483027855,False
149997,35001,headphones,27.0,46838046100,False
149998,35001,headphones,28.0,27242799127,False


## Active Learning (to be moved)

In [28]:
sdbn = sessions_to_sdbn(sessions,
                        prior_weight=10,
                        prior_grade=0.2)
sdbn.loc['transformers dvd']

/tmp/ipykernel_29/3617411679.py:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_29/3617411679.py:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_29/3617411679.py:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_29/3617411679.py:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_29/3617411679.py:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_29/3617411679.py:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

,clicked,examined,grade,beta_grade
doc_id,,,,
97363560449,677,1946,0.347893,0.347137
97361312804,662,1920,0.344792,0.344041
97361312743,657,1916,0.342902,0.342160
97363455349,664,1937,0.342798,0.342065
97361372389,622,1919,0.324127,0.323484
97363532149,623,1927,0.323300,0.322664
879862003517,37,1698,0.021790,0.022834
93624995012,32,1673,0.019127,0.020202
47875842328,29,1663,0.017438,0.018530


## New helper: show the features for each SDBN entry

This function does the same steps as `ranksvm_ltr` but skips model training, returning just the logged feature vector of each example for this query.

In [29]:
def sdbn_with_features(sdbn, feature_set):
    """Log features alongside sdbn into a dataframe"""
    judgments = sdbn_to_judgments(sdbn)
    judgments_path = 'retrotech_judgments.txt'
    write_judgments(judgments, judgments_path)
    
    # For more on this code, review Chapter 10
    requests.delete(f"{SOLR_URL}/products/schema/feature-store/explore")
    
    resp = requests.put(f"{SOLR_URL}/products/schema/feature-store",
                    json=feature_set)

    ftr_logger=FeatureLogger(client, index='products', feature_set='explore', id_field='upc')
    
    with judgments_open(judgments_path) as judgment_list:
        for qid, query_judgments in groupby(judgments, key=lambda j: j.qid):
            ftr_logger.log_for_qid(judgments=query_judgments, 
                                   qid=qid,
                                   keywords=judgment_list.keywords(qid))

    logged_judgments = ftr_logger.logged
    means, std_devs, normed_judgments = normalize_features(logged_judgments)
    feature_deltas, predictor_deltas = pairwise_transform(normed_judgments)
    features, predictors = judgments_to_nparray(logged_judgments)
    logged_judgments_dataframe = pd.concat([pd.DataFrame(predictors),
                                            pd.DataFrame(features)], 
                                           axis=1,
                                           ignore_index=True)
    columns = {idx + 2: ftr['name'] for idx, ftr in enumerate(feature_set)}
    columns[0] = 'grade'
    columns[1] = 'qid'
    
    qid_to_query = {}
    for j in logged_judgments:
        qid_to_query[j.qid] = j.keywords
        
    qid_to_query = pd.DataFrame(qid_to_query.values()).reset_index().rename(columns={'index': 'qid', 0: 'query'})
    
    print(qid_to_query)
    logged_judgments_dataframe = logged_judgments_dataframe.rename(columns=columns)
    print(logged_judgments_dataframe)
    logged_judgments_dataframe = logged_judgments_dataframe.merge(qid_to_query, how='left', on='qid')
    cols_order = ['query', 'grade'] + [ftr['name'] for idx, ftr in enumerate(feature_set)]
    logged_judgments_dataframe['grade'] = logged_judgments_dataframe['grade'] / 10.0 
    return logged_judgments_dataframe[cols_order].sort_values('query')

## Raw feature values -> whats missing?
(put in book)

Another way of formulating `presentation_bias` is to look at the kinds of documents not being shown to users, so we can strategically show those to users. Below we show the value of each feature in `feature_set` for each document in the sdbn judgments.

In [30]:

    
    
sdbn = sessions_to_sdbn(sessions,
                        prior_weight=10,
                        prior_grade=0.2)

feature_set = [
    {
      "name" : "long_desc_match",
      "store": "explore",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "longDescription:(${keywords})^=1"
      }
    },
    {
      "name" : "short_desc_match",
      "store": "explore",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "shortDescription:(${keywords})^=1"
      }
    },
    {
      "name" : "name_match",
      "store": "explore",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "name:(${keywords})^=1"
      }
    },
    {
      "name" : "has_promotion",
      "store": "explore",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "promotion_b:true"
      }
    },
]


sdbn_ftrs = sdbn_with_features(sdbn, feature_set)
transformers_dvds = sdbn_ftrs[sdbn_ftrs['query'] == 'transformers dvd']
transformers_dvds

/tmp/ipykernel_29/3617411679.py:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_29/3617411679.py:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_29/3617411679.py:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_29/3617411679.py:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_29/3617411679.py:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_29/3617411679.py:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

{!terms f=upc}27242815414,600603132872,827396513927,600603141003,885170033412,883929140855,24543672067,813774010904,36725617605,786936817232,36725608443,719192580374,25192073007,75993997675,36725608894,786936817218,711719983156,22265052211,883929197965,25192107191,22265004517,58231306590,58231300826,23942972389,826663129342,885170038875,786936805017,711719804604,36725608511,27242809710
Searching products [Status: 200]
Missing doc 600603132872
Missing doc 600603141003
{!terms f=upc}600603141003,27242813908,600603132872,36725608894,25192107191,36725608511,605342041546,883929140855,786936817232,22265004517,719192580374,24543742180,36725608443,23942973416,36725617605,27242815414,826663129342,58231306590,27242809758,36725608436,36725617506,711719983156,186156000290,58231300826,23942972389,786936805017,25192073007,885170038875,711719804604,27242809710
Searching products [Status: 200]
Missing doc 600603141003
Missing doc 600603132872
{!terms f=upc}856751002097,48231011396,84691226727,74108007

,query,grade,long_desc_match,short_desc_match,name_match,has_promotion
618,transformers dvd,0.0,1.0,0.0,1.0,0.0
623,transformers dvd,0.0,1.0,1.0,1.0,0.0
622,transformers dvd,0.0,1.0,1.0,1.0,0.0
621,transformers dvd,0.0,1.0,0.0,1.0,0.0
620,transformers dvd,0.0,1.0,0.0,1.0,0.0
619,transformers dvd,0.0,1.0,0.0,1.0,0.0
617,transformers dvd,0.0,0.0,0.0,1.0,0.0
610,transformers dvd,0.3,0.0,0.0,1.0,0.0
615,transformers dvd,0.3,0.0,0.0,1.0,0.0
614,transformers dvd,0.3,0.0,0.0,1.0,0.0


## Train Gaussian Process Regressor

We train data on just the `transformers_dvd` training data. 

NOTE we could also train on the full sdbn data, and see globally what's missing. However it's often convenient to zero in on specific queries to round out their training data.

In [31]:
from sklearn.gaussian_process import GaussianProcessRegressor

y_train = transformers_dvds['grade']
x_train = transformers_dvds[['long_desc_match', 'short_desc_match',
                             'name_match', 'has_promotion']]

gpr=GaussianProcessRegressor()
gpr.fit(x_train, y_train)

GaussianProcessRegressor()

## Listing 12.7: Predict on every value

Here `gpr` predicts on every possible feature value. This lets us analyze which set of feature values to use when exploring with users.

In [32]:
zero_or_one = [0,1]

index = pd.MultiIndex.from_product([zero_or_one] * 4,
                                   names = ['long_desc_match', 'short_desc_match',
                                             'name_match', 'has_promotion'])
explore_options = pd.DataFrame(index=index).reset_index()

predictions_with_std = gpr.predict(explore_options[['long_desc_match', 'short_desc_match',
                                                    'name_match', 'has_promotion']], return_std=True)
explore_options['predicted_grade'] = predictions_with_std[0]
explore_options['prediction_stddev'] = predictions_with_std[1]

explore_options.sort_values('prediction_stddev')

,long_desc_match,short_desc_match,name_match,has_promotion,predicted_grade,prediction_stddev
2,0,0,1,0,2.250004e-01,0.000004
10,1,0,1,0,1.192093e-07,0.000004
14,1,1,1,0,0.000000e+00,0.000007
6,0,1,1,0,1.192093e-07,0.000010
0,0,0,0,0,1.364695e-01,0.795060
3,0,0,1,1,1.364695e-01,0.795060
8,1,0,0,0,0.000000e+00,0.795060
11,1,0,1,1,0.000000e+00,0.795060
12,1,1,0,0,0.000000e+00,0.795060
15,1,1,1,1,0.000000e+00,0.795060


## Calculate Expected Improvement

We use [Expected Improvement](https://distill.pub/2020/bayesian-optimization/) scoring to select candidates for exploration within the `transformers dvd` query.

In [33]:
from scipy.stats import norm


theta = 0.6
explore_options['opportunity'] = explore_options['predicted_grade'] - sdbn['grade'].mean() - theta


explore_options['prob_of_improvement'] = norm.cdf( (explore_options['opportunity']) / explore_options['prediction_stddev'])

explore_options['expected_improvement'] = explore_options['opportunity'] * explore_options['prob_of_improvement'] \
 + explore_options['prediction_stddev'] * norm.pdf( explore_options['opportunity'] / explore_options['prediction_stddev'])



explore_options.sort_values('expected_improvement', ascending=False).head()

,long_desc_match,short_desc_match,name_match,has_promotion,predicted_grade,prediction_stddev,opportunity,prob_of_improvement,expected_improvement
1,0,0,0,1,8.277285e-02,0.929873,-0.697673,0.226541,0.121908
5,0,1,0,1,-5.960464e-08,0.929873,-0.780446,0.200650,0.104241
13,1,1,0,1,-5.960464e-08,0.929873,-0.780446,0.200650,0.104241
9,1,0,0,1,-5.960464e-08,0.929873,-0.780446,0.200650,0.104241
0,0,0,0,0,1.364695e-01,0.795060,-0.643976,0.208978,0.093904


In [34]:
sdbn.loc['transformers dvd']

,clicked,examined,grade,beta_grade
doc_id,,,,
97363560449,677,1946,0.347893,0.347137
97361312804,662,1920,0.344792,0.344041
97361312743,657,1916,0.342902,0.342160
97363455349,664,1937,0.342798,0.342065
97361372389,622,1919,0.324127,0.323484
97363532149,623,1927,0.323300,0.322664
879862003517,37,1698,0.021790,0.022834
93624995012,32,1673,0.019127,0.020202
47875842328,29,1663,0.017438,0.018530


In [35]:
explore_vect = explore_options.sort_values('expected_improvement', ascending=False).head().iloc[0][['long_desc_match',
                                                                                     'short_desc_match', 
                                                                                     'name_match',
                                                                                     'has_promotion']]
explore_vect

long_desc_match     0.0
short_desc_match    0.0
name_match          0.0
has_promotion       1.0
Name: 1, dtype: float64

## Create a query to fetch 'explore' docs

Based on the selected features from the GaussianProcessRegressor, we create a query to fetch a doc that contains those features.

In [36]:
def explore_query(explore_vect, query):
    config_explore = {'long_desc_match':  {'field': 'long_description', 'query_dependent': True},
                      'short_desc_match': {'field': 'short_description', 'query_dependent': True},
                      'name_match':       {'field': 'name', 'query_dependent': True},
                      'has_promotion':    {'field': 'promotion_b', 'query_dependent': False, '1_value': 'true'}
                     }
    clauses = []
    for col_name, config in config_explore.items():
        if explore_vect[col_name] == 1.0:
            clause = f"+{config['field']}:"
            if config['query_dependent']:
                clause += f"({query})"
            else:
                clause += f"{config['1_value']}"
            clauses.append(clause)
    return ' '.join(clauses)
    
    # fields = ['long_description', 'short_description', 'name', 'is_promotion']
    # field_value = ['match', 'match', 'match', True]

explore_query(explore_vect, 'transformers dvd')


'+promotion_b:true'

In [37]:
import random
random.seed(1234)

explore_vect = explore_options.sort_values('expected_improvement', ascending=False).head().iloc[0][['long_desc_match',
                                                                                     'short_desc_match', 
                                                                                     'name_match',
                                                                                     'has_promotion']]

def explore(query, explore_vect, log=False):
    """ Explore according to the provided explore vector, select
        a random doc from that group."""
    solr_query = explore_query(explore_vect, query)
    
    draw = random.random()

    request = {
            "fields": ["upc", "name", "manufacturer", "score"],
            "limit": 1,
            "params": {
              "q": solr_query,
              "sort": f"random_{draw} DESC"
            }
        }
    
    resp = requests.post(f"{SOLR_URL}/products/select", 
                                   json=request).json()
    
    return resp['response']['docs'][0]['upc']

explore_upc = explore('transformers dvd', explore_vect)
explore_upc

'600603135088'

## Simulate new sessions with the new data

We simulate new sessions, if the upc is in `might_purchase` or `wants_to_purchase`, we set it to 'clicked' with a given probability.

In [ ]:
import random
random.seed(1234)

wants_to_purchase = ['97360724240', '97363560449', '97363532149', '97360810042', '97368920347']
might_purchase = ['97361312743', '97363455349', '97361372389']
explore_on_rank = 2.0

with_explore_sessions = sessions.copy()
for i in range(0, 500):
    print(i)
    explore_upc = explore('transformers dvd', explore_vect)
    sess_ids = list(set(sessions[sessions['query'] == 'transformers dvd']['sess_id'].tolist()))
    random.shuffle(sess_ids)
    sess_ids[0]
    new_session = sessions[sessions['sess_id'] == sess_ids[0]].copy()
    new_session['sess_id'] = 100000 + i
    new_session.loc[new_session['rank'] == explore_on_rank, 'doc_id'] = explore_upc
    draw = random.random()
    new_session.loc[new_session['rank'] == explore_on_rank, 'clicked'] = False
    if explore_upc in wants_to_purchase:
        if draw < 0.8:
            print(f"click {explore_upc}")
            new_session.loc[new_session['rank'] == explore_on_rank, 'clicked'] = True
    elif explore_upc in might_purchase:
        if draw < 0.5:
            print(f"click {explore_upc}")
            new_session.loc[new_session['rank'] == explore_on_rank, 'clicked'] = True
    else:
        if draw < 0.01:
            print(f"click {explore_upc}")
            new_session.loc[new_session['rank'] == explore_on_rank, 'clicked'] = True

    with_explore_sessions = pd.concat([with_explore_sessions, new_session])

with_explore_sessions

0
1
click 97360810042
2
3
click 97360810042
4
click 36725236271
5
click 97360810042
6
click 97368920347
7
8
9
10
11
click 97368920347
12
13
14
click 97360724240
15
16
17
18
19
click 97360810042
20
click 97360810042
21
22
23
click 97360724240
24
25
26
27
28
29
30
31
32
33
34
35
click 97368920347
36
click 97360722345
37
38
click 97368920347
39
40
41
42
click 97360810042
43
44
45
46
47
click 97360810042
48
49
click 97360810042
50
click 97368920347
51
52
click 97360810042
53
54
55
click 97360810042
56
click 97360810042
57
58
click 97368920347
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
click 97360810042
84
85
86
87
88
89
90
91
92
click 97368920347
93
94
95
click 97360810042
96
97
click 97360724240
98
99
click 97368920347
100
101
102
103
click 97368920347
104
105
click 97360724240
106
107
click 97360724240
108
109


In [ ]:
with_explore_sessions[with_explore_sessions['sess_id'] == 100049]

## Examine new sessions

Have we added any new docs that appear to be getting more clicks?

In [ ]:
new_sdbn = sessions_to_sdbn(with_explore_sessions,
                            prior_weight=10,
                            prior_grade=0.2)
new_sdbn.loc['transformers dvd']

## New heavily clicked doc is promoted!

```
      {
        "upc":"97360810042",
        "name":"Transformers: Dark of the Moon - Blu-ray Disc",
        "name_ngram":"Transformers: Dark of the Moon - Blu-ray Disc",
        "name_omit_norms":"Transformers: Dark of the Moon - Blu-ray Disc",
        "name_txt_en_split":"Transformers: Dark of the Moon - Blu-ray Disc",
        "manufacturer":"\\N",
        "shortDescription":"\\N",
        "longDescription":"\\N",
        "promotion_b":true,
        "id":"72593b1c-313b-4f25-a4f2-04eae29d858b",
        "_version_":1710117636920049669
      },
```

In [ ]:
random.seed(1234)

# {'blue ray': 0.0,
# 'dryer': 0.07068309073137659,
# 'headphones': 0.06426395939086295,
# 'dark of moon': 0.25681268708548055,
# 'transformers dvd': 0.10077083021678328}

feature_set_better = [
    {
      "name" : "name_fuzzy",
      "store": "test",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : { 
        "q" : "name_ngram:(${keywords})"
      }
    },
    {
      "name" : "name_pf2",
      "store": "test",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : { 
        "q" : "{!edismax qf=name name pf2=name}(${keywords})"
      }
    },
    {
      "name" : "shortDescription_pf2",
      "store": "test",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : { 
        "q" : "{!edismax qf=shortDescription pf2=shortDescription}(${keywords})"
      }
    },
    {
      "name" : "has_promotion",
      "store": "test",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "promotion_b:true^=1.0"
      }
    }
]

train, test = test_train_split(new_sdbn, train=0.8)
ranksvm_ltr(train, model_name='test3', feature_set=feature_set_better)
eval_model(test, model_name='test3', sdbn=new_sdbn)

In [ ]:
test

In [ ]:
eval_model(test, model_name='test3', sdbn=new_sdbn)

In [ ]:
new_transformers = new_sdbn.loc['transformers dvd']
new_transformers.loc['97368920347']

In [ ]:
search('transformers dvd', 'test3', at=5)

In [ ]:
NUM_USERS=1000
purchases = {'test1': 0, 'test3': 0}
for _ in range(0, NUM_USERS):
    
    model_name, purchase_made = a_or_b_model(query='transformers dvd', 
                                             a_model='test1',
                                             b_model='test3')
    if purchase_made:
        purchases[model_name]+= 1 
    
purchases

In [ ]:

# Full, auto-exploring, auto LTR loop

# OFFLINE 
# Periodically - Train a model given current info
# Same time -> Generate explore options



# ONLINE

# return results of current model
# mix in random upcs that match the explore criteria
explore_upc = explore('transformers dvd', explore_vect)